**Задание**

**Цель:**

Изучить методы отбора признаков для эффективного обучения моделей машинного обучения.

**Описание задания:**

Решить задачу классификации точек наиболее эффективно. Для этого в работе необходимо применить различные методы по отбору признаков. Отбор признаков предпочтительнее осуществлять основываясь на математическом аппарате, поэтому данные для этого задания будут сгенерированы, чтобы избежать признаков с физическим смыслом.

**Этапы работы:**

1. Сгенерируйте данные с помощью кода:
from sklearn.datasets import make_classification
x_data_generated, y_data_generated = make_classification(scale=1)

2. Постройте модель логистической регрессии и оцените среднюю точность. Для этого используйте следующий код:
cross_val_score(LogisticRegression(), x, y, scoring=‘accuracy’).mean()

3. Используйте статистические методы для отбора признаков:

  a) Выберите признаки на основе матрицы корреляции.

  b) Отсеките низковариативные признаки (VarianceThreshold).

  c) Повторите п. 2 на отобранных признаках в п. 3a, п. 3b.

4. Осуществите отбор признаков на основе дисперсионного анализа:

  a) Выберите 5 лучших признаков с помощью скоринговой функции для классификации f_classif (SelectKBest(f_classif, k=5)).

  b) Повторите п. 2 на отобранных признаках.

5. Отбор с использованием моделей:

  a) Реализуйте отбор признаков с помощью логистической регрессии. Отобранные признаки подайте далее на вход в саму логистическую регрессию (SelectFromModel). Используйте L1 регуляризацию.

  b) Реализуйте отбор признаков с помощью модели RandomForest и встроенного атрибута feature_impotance.

  c) Повторите п. 2 на отобранных признаках в п. 5a, п. 5b.

6. Перебор признаков:

  a) SequentialFeatureSelector.

  b) Повторите п. 2 на отобранных признаках.

7. Сформулируйте выводы по проделанной работе:
  a) Сделайте таблицу вида |способ выбора признаков|количество признаков|средняя точность модели|.



**1. Сгенерируйте данные с помощью кода:**

`from sklearn.datasets import make_classification`

`x_data_generated, y_data_generated = make_classification(scale=1)`

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.datasets import make_classification

In [ ]:
x_data_generated, y_data_generated = make_classification(scale=1, random_state=9)

In [ ]:
x_data_generated.shape, y_data_generated.shape

((100, 20), (100,))

**2. Постройте модель логистической регрессии и оцените среднюю точность. Для этого используйте следующий код:**

`cross_val_score(LogisticRegression(), x, y, scoring=‘accuracy’).mean()`

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model_logreg = LogisticRegression()

In [ ]:
model_logreg.fit(x_data_generated, y_data_generated)

LogisticRegression()

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cvs_all = cross_val_score(model_logreg, x_data_generated, y_data_generated, scoring='accuracy').mean()
cvs_all

0.86

**3. Используйте статистические методы для отбора признаков:**

  **a) Выберите признаки на основе матрицы корреляции.**

In [ ]:
df_x = pd.DataFrame(x_data_generated)
df_y = pd.DataFrame({'target':y_data_generated})

In [ ]:
df = df_x.join(df_y)
df[:2]

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,target
0,-0.061000,-0.868975,0.043621,-2.757511,-1.304449,0.074567,-1.990860,-1.336837,-0.486052,0.954555,...,0.769176,1.529572,0.338366,-2.621578,1.159243,0.956275,-0.437646,0.347415,0.476281,1
1,-1.058712,-1.282205,0.167144,0.144471,-0.861473,0.158882,1.756368,-1.897736,-0.651007,0.337486,...,0.915471,-1.796172,1.798489,-0.541945,0.236401,-0.176654,-0.503043,-1.063231,-0.854900,1


In [ ]:
corr_matrix = df.corr()
corr_matrix.style.background_gradient(cmap='Blues')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,target
0,1.000000,-0.031400,-0.124719,0.227952,-0.039669,0.008475,0.002980,0.020154,0.059414,0.135268,-0.153262,0.063610,0.179177,0.065036,0.010816,-0.053966,-0.125639,-0.062948,-0.033934,-0.021112,0.161469
1,-0.031400,1.000000,-0.049903,0.098799,-0.004746,-0.215948,0.049646,0.049775,0.250644,-0.032055,-0.035216,-0.175420,0.058578,-0.143990,0.043808,0.048970,0.102816,0.174688,0.044627,0.095289,-0.113391
2,-0.124719,-0.049903,1.000000,0.068601,-0.077628,-0.040216,-0.133043,-0.020705,-0.083956,-0.022989,0.207202,-0.040398,0.065487,-0.054203,0.091816,-0.102191,-0.078314,0.039579,0.039408,-0.067163,-0.088631
3,0.227952,0.098799,0.068601,1.000000,0.123978,-0.196373,0.103752,0.093928,0.060212,0.096490,0.136188,0.068491,0.054319,0.217036,-0.006861,0.054729,0.025833,-0.063234,-0.240047,-0.062766,-0.058947
4,-0.039669,-0.004746,-0.077628,0.123978,1.000000,0.060588,0.111912,-0.080980,0.104629,-0.259849,-0.049722,0.012883,-0.116725,0.026897,0.039045,-0.045360,0.115545,-0.012324,-0.025872,0.029303,-0.066213
5,0.008475,-0.215948,-0.040216,-0.196373,0.060588,1.000000,-0.114471,0.037046,-0.035419,-0.209459,-0.069581,0.173938,-0.041171,0.062412,0.090621,0.098301,-0.028771,-0.175696,0.066996,0.148329,0.136104
6,0.002980,0.049646,-0.133043,0.103752,0.111912,-0.114471,1.000000,-0.108842,0.077029,-0.062414,0.002106,0.091717,0.039247,0.073273,0.081370,-0.066061,-0.019353,-0.091397,-0.020550,0.035800,0.074869
7,0.020154,0.049775,-0.020705,0.093928,-0.080980,0.037046,-0.108842,1.000000,-0.073145,-0.092143,0.000526,-0.087704,0.013267,-0.004582,-0.113739,-0.132013,0.184916,0.089422,-0.071003,-0.011177,-0.062309
8,0.059414,0.250644,-0.083956,0.060212,0.104629,-0.035419,0.077029,-0.073145,1.000000,-0.085400,-0.216015,-0.041755,0.181018,0.008869,0.062250,0.058623,0.097774,0.042915,-0.049101,0.008735,-0.030969
9,0.135268,-0.032055,-0.022989,0.096490,-0.259849,-0.209459,-0.062414,-0.092143,-0.085400,1.000000,0.124372,-0.097175,0.091153,0.003595,-0.082467,-0.005388,0.055199,0.099347,-0.090675,-0.039976,-0.065364


Согласно матрице корреляции, уберем признаки, модуль коэффициента корреляции которых меньше 0,1.

In [ ]:
# Отсеивание наименее значимых признаков:
df_1 = df.loc[:, [0, 1, 5, 10, 11, 13, 15, 17]]
df_1.head()

,0,1,5,10,11,13,15,17
0,-0.061000,-0.868975,0.074567,-0.839500,0.769176,0.338366,1.159243,-0.437646
1,-1.058712,-1.282205,0.158882,-0.196260,0.915471,1.798489,0.236401,-0.503043
2,1.101442,-1.398838,1.336861,-1.526132,1.100319,1.818105,0.430021,-0.609006
3,0.267982,0.357477,0.319371,0.725890,0.219119,2.415371,-0.622297,-0.095031
4,-0.621145,0.470908,0.121555,2.216421,0.075988,1.226548,1.754514,-0.027984


In [ ]:
# Датасет с признаками и серия с таргетом:
X_1 = df_1
y_1 = df.target

  **b) Отсеките низковариативные признаки (VarianceThreshold).**

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
selector_VTh = VarianceThreshold(threshold=0.5)
X_VTh = pd.DataFrame(selector_VTh.fit_transform(X_1))
X_VTh[:3]

,0,1,2,3,4,5,6
0,-0.061000,-0.868975,0.074567,-0.839500,0.769176,0.338366,1.159243
1,-1.058712,-1.282205,0.158882,-0.196260,0.915471,1.798489,0.236401
2,1.101442,-1.398838,1.336861,-1.526132,1.100319,1.818105,0.430021


В результате отбора осталось 7 признаков.

  **c) Повторите п. 2 на отобранных признаках в п. 3a, п. 3b.**

In [ ]:
model_logreg_1 = LogisticRegression()
model_logreg_1.fit(X_VTh, y_1)
cvs_stat = cross_val_score(model_logreg_1, X_VTh, y_1, scoring='accuracy').mean()
cvs_stat

0.89

**4. Осуществите отбор признаков на основе дисперсионного анализа:**

  **a) Выберите 5 лучших признаков с помощью скоринговой функции для классификации f_classif (SelectKBest(f_classif, k=5)).**

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [ ]:
selector_KB = SelectKBest(f_classif, k=5)
X_KB = pd.DataFrame(selector_KB.fit_transform(x_data_generated, y_data_generated))
X_KB[:3]

,0,1,2,3,4
0,-0.061000,-0.839500,0.769176,0.338366,-0.437646
1,-1.058712,-0.196260,0.915471,1.798489,-0.503043
2,1.101442,-1.526132,1.100319,1.818105,-0.609006


  **b) Повторите п. 2 на отобранных признаках.**

In [ ]:
model_logreg_2 = LogisticRegression()
model_logreg_2.fit(X_KB, y_1)
cvs_disp = cross_val_score(model_logreg_2, X_KB, y_1, scoring='accuracy').mean()
cvs_disp

0.89

**5. Отбор с использованием моделей:**

  **a) Реализуйте отбор признаков с помощью логистической регрессии. Отобранные признаки подайте далее на вход в саму логистическую регрессию (SelectFromModel). Используйте L1 регуляризацию.**

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Отбор 5-ти признаков с момощью логистической регрессии и L1-регуляризации:
model_logreg_L1 = LogisticRegression(penalty='l1', solver='liblinear')
selector_SfM = SelectFromModel(model_logreg_L1, max_features=5)
X_SfM = pd.DataFrame(selector_SfM.fit_transform(x_data_generated, y_data_generated))
X_SfM[:3]

,0,1,2,3,4
0,-1.304449,-0.839500,0.769176,1.159243,0.956275
1,-0.861473,-0.196260,0.915471,0.236401,-0.176654
2,1.249654,-1.526132,1.100319,0.430021,0.341622


  **b) Реализуйте отбор признаков с помощью модели RandomForest и встроенного атрибута feature_impotance.**

In [ ]:
model_RF = RandomForestClassifier(random_state=9)
model_RF.fit(x_data_generated, y_data_generated)
model_RF.feature_importances_

array([0.02491007, 0.0102149 , 0.02059889, 0.01576246, 0.03008207,
       0.01936536, 0.01624872, 0.01891423, 0.01824837, 0.01544424,
       0.0196246 , 0.27070662, 0.01300673, 0.11890757, 0.01900475,
       0.02260322, 0.01654068, 0.27567346, 0.0312303 , 0.02291276])

Добавим номера признаков в соответствии с именами колонок исходного датафрейма (df_x) и отсортируем их по степени важности.

In [ ]:
importance = pd.Series(model_RF.feature_importances_,index=df_x.columns).sort_values(ascending=False)
importance

17    0.275673
11    0.270707
13    0.118908
18    0.031230
4     0.030082
0     0.024910
19    0.022913
15    0.022603
2     0.020599
10    0.019625
5     0.019365
14    0.019005
7     0.018914
8     0.018248
16    0.016541
6     0.016249
3     0.015762
9     0.015444
12    0.013007
1     0.010215
dtype: float64

Оставим топ-3 первых по важности признака.

  **c) Повторите п. 2 на отобранных признаках в п. 5a, п. 5b.**

In [ ]:
# Модель лог.регрессии и ее метрика для п. 5,а):
model_logreg.fit(X_SfM, y_data_generated)
cvs_SfM = cross_val_score(model_logreg, X_SfM, y_data_generated, scoring='accuracy').mean()
cvs_SfM

0.9099999999999999

In [ ]:
# Модель лог.регрессии и ее метрика для п. 5,b):
model_logreg.fit(df_x.loc[:, [7, 11, 13]], y_data_generated )
csv_RF = cross_val_score(model_logreg, df_x.loc[:, [7, 11, 13]], y_data_generated, scoring='accuracy').mean()
csv_RF

0.89

**6. Перебор признаков:**

  **a) SequentialFeatureSelector.**

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

In [ ]:
sfs = SequentialFeatureSelector(RandomForestClassifier(random_state=9), n_features_to_select=6, direction='forward')
sfs.fit(df_x, y_data_generated)

SequentialFeatureSelector(estimator=RandomForestClassifier(random_state=9),
                          n_features_to_select=6)

In [ ]:
X_SFS = sfs.transform(df_x)
X_SFS.shape

(100, 6)

**b) Повторите п. 2 на отобранных признаках.**

In [ ]:
model_logreg.fit(X_SFS, y_data_generated)
csv_sfs = cross_val_score(model_logreg, X_SFS, y_data_generated, scoring='accuracy').mean()
csv_sfs

0.8799999999999999

**7. Сформулируйте выводы по проделанной работе:**

  **a) Сделайте таблицу вида |способ выбора признаков|количество признаков|средняя точность модели|.**

In [ ]:
pd.DataFrame({'Способ выбора признаков':['До выбора', 'Стат. методы', 'Дисперсионный анализ', 'SelectFromModel+L1', 'RandomForest+feature_impotance', 'SequentialFeatureSelector'],
             'Количество признаков':[x_data_generated.shape[1], X_VTh.shape[1], X_KB.shape[1], X_SfM.shape[1], 3, X_SFS.shape[1]],
              'Средняя точность модели': [cvs_all, cvs_stat, cvs_disp, cvs_SfM, csv_RF, csv_sfs]})

,Способ выбора признаков,Количество признаков,Средняя точность модели
0,До выбора,20,0.86
1,Стат. методы,7,0.89
2,Дисперсионный анализ,5,0.89
3,SelectFromModel+L1,5,0.91
4,RandomForest+feature_impotance,3,0.89
5,SequentialFeatureSelector,6,0.88


До отбора признаков модель обладала самой низкой метрикой качества, все испробованные в работе методы позволили на несколько процентов улучшить метрику, убрав при этом больше половины признаков, что эффектно демонстрирует важность процедуры отбора признаков.